Get packages

In [1]:
import os

os.chdir(os.path.join(os.getcwd(), '..'))
import src.preprocessing
import src.model_training
import src.feature_engineering

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time
import shap
import lightgbm as lgb

import json
import sys
import pickle

from verstack import LGBMTuner
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Get data & model

In [2]:
df = src.feature_engineering.get_train_data('data/data_pricing_challenge.csv')

AttributeError: module 'src.preprocessing' has no attribute 'get_train_data'

In [ ]:
pkl_file_path  = 'models/model_baseline'
# Load the model
with open(pkl_file_path, 'rb') as f:
    model = pickle.load(f)

Prepare data for prediction

In [ ]:
variables = src.model_training.txt_load('utils/model_variables.txt')
df = df.loc[df['price'] >= 20000]
df['mileage'] = df['mileage']+10000
df['antiquity'] = df['antiquity']+1
df_pred = df[variables]
df.shape, df_pred.shape

((999, 39), (999, 19))

In [ ]:
# Make predictions
prediction = model.predict(df_pred)

# Add prediction to dataset
df_pred['original_price'] = df['price']
df_pred['predicted_price'] = prediction
df_pred['loss_in_value'] = df_pred['original_price'] - df_pred['predicted_price']
df_pred = df_pred.sort_values(by='loss_in_value', ascending = True)

In [ ]:
df_pred.head(20)

,mileage,engine_power,antiquity,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,convertible,coupe,estate,hatchback,sedan,subcompact,suv,van,original_price,predicted_price,loss_in_value
3874,81015,190,4.003422,1,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,26900,36241.078407,-9341.078407
4370,144154,300,8.006160,1,1,0,1,1,0,1,1,0,0,0,0,0,0,1,0,22000,29314.220400,-7314.220400
2165,74985,230,3.004107,1,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,34500,40221.088192,-5721.088192
4511,69934,140,4.011636,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,28700,34289.290237,-5589.290237
1508,31507,160,3.017796,1,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,25000,29993.947531,-4993.947531
16,31167,135,3.992471,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,21900,26723.062431,-4823.062431
1984,155497,235,5.991102,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,21700,26368.864228,-4668.864228
3852,71969,135,4.994524,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,25100,29611.113351,-4511.113351
2957,170934,233,5.000000,0,1,1,0,1,1,1,1,0,0,0,0,1,0,0,0,22000,26462.303177,-4462.303177
970,194955,280,6.987680,1,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0,23800,28171.310771,-4371.310771


In [ ]:
df.head()

,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,antiquity,diesel,electro,hybrid_petrol,petrol,beige,black,blue,brown,green,grey,orange,red,silver,white,convertible,coupe,estate,hatchback,sedan,subcompact,suv,van
0,140411,100,2012-01-02,diesel,black,convertible,1,1,0,0,1,1,1,0,11300,2018-01-01,5.998631,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,13929,317,2016-01-04,petrol,grey,convertible,1,1,0,0,0,1,1,1,69700,2018-01-02,1.995893,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,183297,120,2012-01-04,diesel,white,convertible,0,0,0,0,1,0,1,0,10200,2018-01-02,5.995893,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,128035,135,2014-01-07,diesel,red,convertible,1,1,0,0,1,1,1,1,25100,2018-01-02,3.986311,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4,97097,160,2014-01-12,diesel,silver,convertible,1,1,0,0,0,1,1,1,33400,2018-01-04,3.978097,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
